# Categorise the witnesses and investigate the distribution

#### This script aggregates the witnesses industries into research relevant categories and investigates the distribution

In [1]:
from TextCollection import *

In [2]:
# # Reinstantiate class after changing the Textcollection.py script
# # Do not run this in the last run! Leads to a conflict with pickle.

# from importlib import reload

# os.chdir('/home/mirjam/OneDrive/congress_committees/ArticleOne/Article_Scripts/Hearings')
# import TextCollection; reload(TextCollection)

# # Reinstantiate class
# t.__class__ = HearingsCollection

In [3]:
# Change directory
os.chdir('../../Data/')

In [4]:
# Run for maximum recursion error during saving

import resource
import sys

print(resource.getrlimit(resource.RLIMIT_STACK))
print(sys.getrecursionlimit())

# May segfault without this line. 0x100 is a guess at the size of each stack frame.
max_rec = 0x100000
resource.setrlimit(resource.RLIMIT_STACK, [0x100 * max_rec, resource.RLIM_INFINITY])
sys.setrecursionlimit(max_rec)

print(sys.getrecursionlimit())

(8388608, -1)
3000
1048576


<br>

## 1) Preparation: Loading, correcting and inspecting the data

In [5]:
t = load('Hearings/06_witnesses.pkl')
with open('OpenSecrets/sectors_industries_contributors.json', 'r') as jfile:
    sectors = json.load(jfile)
print('We imported {} hearings and lobbying {} sectors for the years {} to {}.\n'.format(len(t),len(sectors), sectors[0]['industries'][0]['year'][0], 
                                                                                                        sectors[0]['industries'][0]['year'][-1]))
# Append sectors to hearings collection
t.sectors = sectors

We imported 257 hearings and lobbying 13 sectors for the years 2003 to 2010.



In [6]:
count = 0
for sector in t.sectors:
    for industry in sector['industries']:
        count += 1
        print(count, industry['industry'])
        
print('\n', count)

1 Agricultural Services/Products
2 Crop Production & Basic Processing
3 Dairy
4 Farm bureaus
5 Food and kindred products manufacturing
6 Food Processing & Sales
7 Food stores
8 Forestry & Forest Products
9 Livestock
10 Meat processing & products
11 Poultry & Eggs
12 Sugar cane & sugar beets
13 Tobacco
14 Vegetables, fruits and tree nut
15 Book, newspaper & periodical publishing
16 Cable & satellite TV production
17 Commercial TV & radio stations
18 Computer software
19 Electronics Mfg & Equip
20 Internet
21 Motion Picture production & distribution
22 Printing & Publishing
23 Recorded Music & music production
24 Telecom Services
25 Telephone Utilities
26 TV production
27 TV/Movies/Music
28 Architectural services
29 Building Materials & Equipment
30 Construction Services
31 General Contractors
32 Home Builders
33 Special Trade Contractors
34 Defense Aerospace
35 Defense Electronics
36 Misc Defense
37 Alternate energy production & services
38 Coal mining
39 Electric Utilities
40 Mining
41

In [7]:
t.texts[0].keys()

dict_keys(['filename', 'identifier', 'content_raw', 'content', 'content_stripped', 'congress', 'climate_related', 'keywords_climate', 'keywords_carbonpricing', 'keywords_climatechange', 'title', 'date', 'year', 'committee', 'MODS', 'witnesses', 'desmog_witness', 'desmog_organisation', 'witness_affiliation', 'witness_sector', 'witness_industry', 'state_research', 'contrarian_organisation'])

<br>

## 2) Extract contrarian witnesses

In [8]:
# Create variables that class witnesses as contrarian if they are einther know contrarians
# or are affiliated with a contrarian organisation

# DeSmog or not?
desmog_witnesses = 0
for text in t.texts:
    text['desmog'] = []
    for i, witness in enumerate(text['witnesses']):
        if text['desmog_witness'][i] != None or text['desmog_organisation'][i] != None:
            text['desmog'].append('DeSmog witness') 
            desmog_witnesses += 1
        else:
            text['desmog'].append('Other witness') 
print("{} desmog witnesses were found.".format(desmog_witnesses))

# Contrarian or not?
contrarian_witnesses = 0
for text in t.texts:
    text['contrarian'] = []
    for i, witness in enumerate(text['witnesses']):
        if text['desmog'][i] == 'DeSmog witness' or text['contrarian_organisation'][i] != None:
            text['contrarian'].append('Contrarian') 
            contrarian_witnesses += 1
        else:
            text['contrarian'].append('Other witness') 
print("{} contrarian witnesses were found.".format(contrarian_witnesses))

109 desmog witnesses were found.
130 contrarian witnesses were found.


<br>

## 3) Aggregate the witnesses

### Level 1 aggregation: subsubcategories

In [9]:
# Aggregate the witnesses from the opensecrets affiliations into subsubcategories

subsubcategories = []
for text in t.texts:
    text['subsubcategory'] = []
    for i, witness_sector in enumerate(text['witness_sector']):
        if (witness_sector in ['Agribusiness', 'Communications/Electronics', 'Construction', 'Finance, Insurance & Real Estate', 
                              'Health', 'Labor', 'Lawyers & Lobbyists', 'Misc Business', 'Transportation'] 
            and text['witness_industry'][i] not in ['Steel Production', 'Chemical & Related Manufacturing']):
            text['subsubcategory'].append(witness_sector) 
            subsubcategories.append(witness_sector)
        else:
            text['subsubcategory'].append(text['witness_industry'][i])
            subsubcategories.append(text['witness_industry'][i])
            
subsubcategories = set(subsubcategories)

print("Witnesses with the following affiliations were found:")
subsubcategories

Witnesses with the following affiliations were found:


{'Agribusiness',
 'Alternate energy production & services',
 'Chemical & Related Manufacturing',
 'Civil Servants/Public Officials',
 'Clergy & Religious Organizations',
 'Coal mining',
 'Communications/Electronics',
 'Construction',
 'Defense Aerospace',
 'Defense Electronics',
 'Democratic/Liberal',
 'Education',
 'Electric Utilities',
 'Environment',
 'Finance, Insurance & Real Estate',
 'Foreign & Defense Policy',
 'Health',
 'Human Rights',
 'International/Intergovernmental',
 'Labor',
 'Lawyers & Lobbyists',
 'Mining',
 'Misc Business',
 'Misc Defense',
 'Natural Gas transmission & distribution',
 'Non-Profit Institutions',
 'Oil & Gas',
 'Republican/Conservative',
 'Retired',
 'Steel Production',
 'Transportation',
 'Waste Management',
 "Women's Issues"}

In [10]:
# Change subsubcategory of nuclear electric utilities to alternate energy production

count=0
for text in t.texts:
    for i, affiliation in enumerate(text['witness_affiliation']):
        if len(re.findall('nuclear energy|nrg energy|nuclear society|nuclear$',
                          affiliation.lower())) > 0:
            print(affiliation)
            text['subsubcategory'][i] = 'Alternate energy production & services'
            count += 1

print('\n{} nuclear witnesses were recategorised as "Alternate energy production & services"'.format(count))            

NRG Energy
Ge Hitachi Nuclear
NRG Energy
Nuclear Energy Institute
NRG Energy
NRG Energy
Babcock & Wilcox Nuclear
American Nuclear Society
Nuclear Energy Institute
Nuclear Energy Institute
Nuclear Energy Institute
NRG Energy

12 nuclear witnesses were recategorised as "Alternate energy production & services"


### Level 2 aggregation: subcategories

In [11]:
# Aggregate the witnesses from the subsubcategories into subcategories

subcategories = []
for text in t.texts:
    text['subcategory'] = []
    for i, subsubcategory in enumerate(text['subsubcategory']):
        if text['contrarian'][i] == 'Contrarian':
            text['subcategory'].append('Contrarians') 
            subcategories.append('Contrarians')
            
        elif subsubcategory in ['Republican/Conservative']:
            text['subcategory'].append(subsubcategory) 
            subcategories.append(subsubcategory)
            
        elif subsubcategory in ['Alternate energy production & services']:
            text['subcategory'].append('Alternate Energy Production & Services') 
            subcategories.append('Alternate Energy Production & Services')
            
        elif subsubcategory in ['Agribusiness', 'Chemical & Related Manufacturing', 'Construction', 'Defense Aerospace',
                                 'Defense Electronics', 'Mining', 'Misc Defense', 'Steel Production', 'Transportation', 'Waste Management']:
            text['subcategory'].append('Carbon-intensive Industry') 
            subcategories.append('Carbon-intensive Industry')
            
        elif subsubcategory in ['Environment']:
            text['subcategory'].append('Environmental Allies') 
            subcategories.append('Environmental Allies')
            
        elif subsubcategory in ['Coal mining', 'Electric Utilities', 'Natural Gas transmission & distribution', 'Oil & Gas']:
            text['subcategory'].append('Fossil Fuel Industry') 
            subcategories.append('Fossil Fuel Industry')
            
        elif subsubcategory in ['Civil Servants/Public Officials']:
            text['subcategory'].append('Government Officials') 
            subcategories.append('Government Officials')
            
        elif subsubcategory in ['Communications/Electronics', 'Health', 'Finance, Insurance & Real Estate', 'Lawyers & Lobbyists', 'Misc Business']:
            text['subcategory'].append('Miscellaneous Business')     
            subcategories.append('Miscellaneous Business') 
            
        elif subsubcategory in ['Democratic/Liberal', 'Human Rights', 'Labor', 'Non-Profit Institutions', "Women's Issues", 'Foreign & Defense Policy']:
            text['subcategory'].append('Non-Profits/Unions') 
            subcategories.append('Non-Profits/Unions') 
            
        elif subsubcategory in ['Clergy & Religious Organizations', 'International/Intergovernmental', 'Retired']:
            text['subcategory'].append('Other') 
            subcategories.append('Other')
            
        elif subsubcategory in ['Education']:
            text['subcategory'].append('Scientists') 
            subcategories.append('Scientists')
            
        else:
            text['subcategory'].append(subsubcategory) 
            subcategories.append(subsubcategory) 
            
subcategories = set(subcategories)

print("Witnesses with the following subcategories were found:")
subcategories

Witnesses with the following subcategories were found:


{'Alternate Energy Production & Services',
 'Carbon-intensive Industry',
 'Contrarians',
 'Environmental Allies',
 'Fossil Fuel Industry',
 'Government Officials',
 'Miscellaneous Business',
 'Non-Profits/Unions',
 'Other',
 'Scientists'}

### Level 3 aggregation: categories

In [12]:
# Aggregate the witnesses from the subcategories into categories

categories = []
for text in t.texts:
    text['category'] = []
    for i, affiliation in enumerate(text['subcategory']):
        if text['contrarian'][i] == 'Contrarian':
            text['category'].append('Contrarians') 
            categories.append('Contrarians')
            
        elif affiliation in ['Government Officials']:
            text['category'].append('Government Officials') 
            categories.append('Government Officials')
            
        elif affiliation in ['Alternate Energy Production & Services', 'Carbon-intensive Industry', 'Fossil Fuel Industry', 'Miscellaneous Business']:
            text['category'].append('Industry Allies') 
            categories.append('Industry Allies')
            
        elif affiliation in ['Democratic/Liberal', 'Environmental Allies', 'Non-Profits/Unions', ]:
            text['category'].append('Non-Governmental Organisations') 
            categories.append('Non-Governmental Organisations')
            
        elif affiliation in ['Scientists']:
            text['category'].append('Scientists') 
            categories.append('Scientists')
            
        elif affiliation in ['Other']:
            text['category'].append('Other') 
            categories.append('Other')
            
categories = set(categories)

print("Witnesses with the following categories were found:")
categories

Witnesses with the following categories were found:


{'Contrarians',
 'Government Officials',
 'Industry Allies',
 'Non-Governmental Organisations',
 'Other',
 'Scientists'}

<br>

## 3) Sankey plots to illustrate witness aggregation and DeSmog wittness affiliation

In [13]:
import pandas as pd
import numpy as np
%matplotlib inline
import plotly.graph_objs as go
from  plotly.offline import plot
import chart_studio.plotly as py
import cufflinks as cf
cf.go_offline()
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [14]:
# Create a dictionary of the sankey plot data (witness_industry to subsubcategory)
sankey_raw_1 = {'witness_industry': [industry for t in t.texts for industry in t['witness_industry']],
                'subsubcategory': [subsubcategory for t in t.texts for subsubcategory in t['subsubcategory']]}
# Transform dictionary to dataframe
df1 = pd.DataFrame.from_dict(sankey_raw_1, orient='columns', dtype=None, columns=None)
# Aggregate and count the rows
df1 = df1[['witness_industry', 'subsubcategory']].groupby(['witness_industry', 'subsubcategory']).size().reset_index(name='count')
# Map id to witness_industries
witness_industries_numeric = range(len(df1.witness_industry.unique()))
witness_industries_dict = dict(zip(sorted(set(df1['witness_industry'])), witness_industries_numeric))
# Map id to subsubcategory
subsubcategories_numeric = range(witness_industries_numeric[-1] + 1, 
                                 witness_industries_numeric[-1] + 1 + len(df1.subsubcategory.unique()))
subsubcategories_dict = dict(zip(sorted(set(df1['subsubcategory'])), subsubcategories_numeric))

df1['source'] = df1['witness_industry'].map(witness_industries_dict)
df1['target'] = df1['subsubcategory'].map(subsubcategories_dict)
df1['color'] = '#262C46'
df1

,witness_industry,subsubcategory,count,source,target,color
0,Agricultural Services/Products,Agribusiness,22,0,73,#262C46
1,Air Transport,Transportation,17,1,103,#262C46
2,Air transport unions,Labor,3,2,92,#262C46
3,Alternate energy production & services,Alternate energy production & services,60,3,74,#262C46
4,Architectural services,Construction,1,4,80,#262C46
...,...,...,...,...,...,...
69,Transportation Unions,Labor,1,68,92,#262C46
70,Trucking,Transportation,5,69,103,#262C46
71,Venture capital,"Finance, Insurance & Real Estate",9,70,87,#262C46
72,Waste Management,Waste Management,7,71,104,#262C46


In [15]:
# Create a dictionary of the sankey plot data (subsubcategory to subcategory)
sankey_raw_2 = {'subsubcategory': [subsubcategory for t in t.texts for subsubcategory in t['subsubcategory']],
                'subcategory': [subcategory for t in t.texts for subcategory in t['subcategory']]}
# Transform dictionary to dataframe
df2 = pd.DataFrame.from_dict(sankey_raw_2, orient='columns', dtype=None, columns=None)
# Aggregate and count the rows
df2 = df2[['subsubcategory', 'subcategory']].groupby(['subsubcategory', 'subcategory']).size().reset_index(name='count')
# Map id to affiliation
subcategories_numeric = range(subsubcategories_numeric[-1] + 1,
                              subsubcategories_numeric[-1] + 1 + len(df2.subcategory.unique()))
subcategories_dict = dict(zip(sorted(set(df2['subcategory'])), subcategories_numeric))

df2['source'] = df2['subsubcategory'].map(subsubcategories_dict)
df2['target'] = df2['subcategory'].map(subcategories_dict)
df2['color'] = '#262C46'
df2

,subsubcategory,subcategory,count,source,target,color
0,Agribusiness,Carbon-intensive Industry,47,73,107,#262C46
1,Agribusiness,Contrarians,4,73,108,#262C46
2,Alternate energy production & services,Alternate Energy Production & Services,72,74,106,#262C46
3,Chemical & Related Manufacturing,Carbon-intensive Industry,4,75,107,#262C46
4,Civil Servants/Public Officials,Contrarians,4,76,108,#262C46
5,Civil Servants/Public Officials,Government Officials,359,76,111,#262C46
6,Clergy & Religious Organizations,Contrarians,1,77,108,#262C46
7,Clergy & Religious Organizations,Other,10,77,114,#262C46
8,Coal mining,Contrarians,3,78,108,#262C46
9,Coal mining,Fossil Fuel Industry,13,78,110,#262C46


In [16]:
# Create a dictionary of the sankey plot data (subcategory to category)
sankey_raw_3 = {'subcategory': [affiliation for t in t.texts for affiliation in t['subcategory']],
                'category': [category for t in t.texts for category in t['category']]}
# Transform dictionary to dataframe
df3 = pd.DataFrame.from_dict(sankey_raw_3, orient='columns', dtype=None, columns=None)
# Aggregate and count the rows
df3 = df3[['subcategory', 'category']].groupby(['subcategory', 'category']).size().reset_index(name='count')
# Map id to affiliation
categories_numeric = range(subcategories_numeric[-1] + 1, 
                         subcategories_numeric[-1] + 1 + len(df3.category.unique()))
categories_dict = dict(zip(sorted(set(df3['category']), reverse = True), categories_numeric))

df3['source'] = df3['subcategory'].map(subcategories_dict)
df3['target'] = df3['category'].map(categories_dict)
df3['color'] = '#262C46'
df3

,subcategory,category,count,source,target,color
0,Alternate Energy Production & Services,Industry Allies,72,106,119,#262C46
1,Carbon-intensive Industry,Industry Allies,153,107,119,#262C46
2,Contrarians,Contrarians,130,108,121,#262C46
3,Environmental Allies,Non-Governmental Organisations,211,109,118,#262C46
4,Fossil Fuel Industry,Industry Allies,163,110,119,#262C46
5,Government Officials,Government Officials,359,111,120,#262C46
6,Miscellaneous Business,Industry Allies,188,112,119,#262C46
7,Non-Profits/Unions,Non-Governmental Organisations,103,113,118,#262C46
8,Other,Other,38,114,117,#262C46
9,Scientists,Scientists,253,115,116,#262C46


In [19]:
# Concatenate the individual dataframes
df = pd.concat([df1,df2,df3])
df['linkcolor'] = '#dee1ed'
df['linkcolor'][df.target == 108] = '#e61931'
df['linkcolor'][df.target == 121] = '#e61931'
node_labels = (sorted([i for i in df1.witness_industry.unique()]) +
               sorted([i for i in df1.subsubcategory.unique()]) + 
               sorted([i for i in df2.subcategory.unique()])  + 
               sorted([i for i in df3.category.unique()], reverse = True))

In [20]:
# Plot the sankey plot
data_trace = dict(
    type='sankey',
    domain = dict(
      x =  [0,1],
      y =  [0,1]
    ),
    orientation = "h",
    valueformat = ".0f",
    node = dict(
      pad = 10,
      thickness = 30,
      line = dict(
        color = "black",
        width = 0.5
      ),
      label =  node_labels,
      color = df['color'],
    ),
    link = dict(
      source = df['source'].dropna(axis=0, how='any'),
      target = df['target'].dropna(axis=0, how='any'),
      value = df['count'].dropna(axis=0, how='any'),
      color = df['linkcolor'].dropna(axis=0, how='any'),
      opacity = 0.4
  )
)

layout =  dict(
    title = "Witness aggregation",
    height = 1200,
    width = 1000,
    font = dict(
      size = 12
    ),    
)

fig = dict(data=[data_trace], layout=layout)
iplot(fig, validate=False)

In [21]:
# Export plot as html
plot(fig, validate=False, filename = '../Plots/sankey_witness_aggregation.html')

'../Plots/sankey_witness_aggregation.html'

<br>

## 4) Sankey plots directly linking witness industries and contrarians

In [22]:
# Create a dictionary of the sankey plot data (levels 0 to 1)
sankey_raw_4 = {'witness_industry': [industry for t in t.texts for industry in t['witness_industry']],
               'contrarian': [contrarian_witness for t in t.texts for contrarian_witness in t['contrarian']]}
# Transform dictionary to dataframe
df4 = pd.DataFrame.from_dict(sankey_raw_4, orient='columns', dtype=None, columns=None)
# Aggregate and count the rows
df4 = df4[['witness_industry', 'contrarian']].groupby(['witness_industry', 'contrarian']).size().reset_index(name='count')
# Map id to witness_industries
witness_industries_numeric = range(len(df4.witness_industry.unique()))
witness_industries_dict = dict(zip(sorted(set(df4['witness_industry'])), witness_industries_numeric))
# witness_industries_dict

# Map id to subsubcategory
contrarians_numeric = range(witness_industries_numeric[-1]+1,
                            witness_industries_numeric[-1]+1 + len(df4.contrarian.unique()))
contrarians_dict = dict(zip(sorted(set(df4['contrarian']), reverse = True), contrarians_numeric))

df4['source'] = df4['witness_industry'].map(witness_industries_dict)
df4['target'] = df4['contrarian'].map(contrarians_dict)
df4['color'] = '#262C46'
df4['linkcolor'] = '#dee1ed'
df4['linkcolor'][df4.target == 74] = '#e61931'
df4

,witness_industry,contrarian,count,source,target,color,linkcolor
0,Agricultural Services/Products,Contrarian,4,0,74,#262C46,#e61931
1,Agricultural Services/Products,Other witness,18,0,73,#262C46,#dee1ed
2,Air Transport,Contrarian,1,1,74,#262C46,#e61931
3,Air Transport,Other witness,16,1,73,#262C46,#dee1ed
4,Air transport unions,Other witness,3,2,73,#262C46,#dee1ed
...,...,...,...,...,...,...,...
85,Transportation Unions,Other witness,1,68,73,#262C46,#dee1ed
86,Trucking,Other witness,5,69,73,#262C46,#dee1ed
87,Venture capital,Other witness,9,70,73,#262C46,#dee1ed
88,Waste Management,Other witness,7,71,73,#262C46,#dee1ed


In [23]:
node_labels4 = ([i for i in df4.witness_industry.unique()] + 
               sorted([i for i in df4.contrarian.unique()], reverse = True))

In [24]:
data_trace = dict(
    type='sankey',
    domain = dict(
      x =  [0,1],
      y =  [0,1]
    ),
    orientation = "h",
    valueformat = ".0f",
    node = dict(
      pad = 10,
      thickness = 30,
      line = dict(
        color = "black",
        width = 0.5
      ),
      label =  node_labels4,
      color = df4['color'],
    ),
    link = dict(
      source = df4['source'].dropna(axis=0, how='any'),
      target = df4['target'].dropna(axis=0, how='any'),
      value = df4['count'].dropna(axis=0, how='any'),
      color = df4['linkcolor'].dropna(axis=0, how='any'),
      opacity = 0.4
  )
)

layout =  dict(
    title = "Contrarian witnesses",
    height = 1400,
    width = 800,
    font = dict(
      size = 10
    ),    
)

fig = dict(data=[data_trace], layout=layout)
iplot(fig, validate=False)

In [25]:
# Export plot as html
plot(fig, validate=False, filename = '../Plots/sankey_contrarians.html')

'../Plots/sankey_contrarians.html'

<br>

## 5) Save and export the witness data

In [26]:
save_as(t, 'Hearings/07_witnesses_aggregated.pkl') # Last completed on March 01, 2022

In [27]:
# 5.1 Export the witnesses
columns = ['congress', 'committee', 'year', 'date', 'hearing_title', 'hearing_id', 'witness', 'affiliation', 
           'opensectrets_industry', 'opensectrets_sector', 'category', 'subcategory', 'subsubcategory',
           'state_research', 'contrarian_organisation', 'contrarian']

In [28]:
export_df = pd.DataFrame(columns=columns)

In [29]:
for text in t.texts:
    for i, witness in enumerate(text['witnesses']):
        print(i, witness, '\n',  text['witness_affiliation'][i])
        export_df = export_df.append({'congress': text['congress'], 'committee': text['committee'],
                                      'year': text['year'], 'date' : text['date'],
                                      'hearing_title': text['title'], 'hearing_id': text['identifier'],
                                      'witness': witness, 'affiliation': text['witness_affiliation'][i],
                                      'opensectrets_industry': text['witness_industry'][i], 
                                      'opensectrets_sector': text['witness_sector'][i],
                                      'category': text['category'][i], 'subcategory': text['subcategory'][i], 
                                      'subsubcategory': text['subsubcategory'][i],
                                      'state_research': text['state_research'][i], 
                                      'contrarian_organisation': text['contrarian_organisation'][i], 
                                      'contrarian': text['contrarian'][i]}, 
                        ignore_index=True)

0 John McMackin, Jr., Williams and Jenson, PLLC, On Behalf Of The Energy Intensive Manufacturers Working Group on Greenhouse Gas Regulations 
 Energy Intensive Manufacturers
1 Martin McBroom, Director, Federal Environmental Affairs, American Electric Power 
 American Electric Power
2 Paul Cicio, Industrial Energy Consumers of America 
 Industrial Energy Consumers of America
3 Margo Thorning, Ph.D., Senior Vice President and Chief Economist, American Council for Capital Formation 
 American Council for Capital Formation
4 Richard D. Morgenstern, Senior Fellow, Resources for the Future 
 Resources for the Future
5 Eileen Claussen, President, Pew Center on Global Climate Change 
 Pew Center on Global Climate Change
0 Thomas Fingar, Deputy Director of National Intelligence for Analysis, Chairman, National Intelligence Council 
 National Intelligence Council
1 Rolf Mowatt-Larssen, Director, Office of Intelligence and Counterintelligence, U.S. Department of Energy 
 Department of Energy
2 Rt

1 James E. Rogers, president, and chief executive officer, Duke Energy Corporation, Charlotte, NC 
 Duke Energy
2 David Sokol, president, and chief executive officer, MidAmerican Corporation, Omaha, ND 
 Midamerican Corporation
3 Michael G. Morris, chairman, president, and chief executive officer, American Electric Power, Columbus, OH 
 American Electric Power
4 Jackson E. Reasor, president, and chief executive officer, Old Dominion Cooperative, Glen Allen, VA 
 Old Dominion Cooperative
5 Milton B. Lee, general manager, and chief executive officer, CPS Energy, San Antonio, TX 
 Cps Energy
0 John B. Stephenson, Director, Natural Resources and Environment, U.S. Government Accountability Office 
 Government Accountability Office
1 Eldon Gould, Administrator, Risk Management Agency, U.S. Department of Agriculture 
 Department of Agriculture
2 Michael Buckley, Deputy Assistant Administrator for Mitigation, Federal Emergency Management Agency, U.S. Department of Homeland Security 
 Departmen

3 Falgout, Ted, Port Director, Port Fourchon, La 
 Port Fourchon
4 Wallace, Terry, Principal Associate Director For Science, Technology And Engineering, Los Alamos National Laboratory, Los Alamos, Nm 
 National Laboratory
5 Wilbanks, Thomas J., Oak Ridge National Laboratory, Oak Ridge, Tn 
 National Laboratory
0 Beisner, E. Calvin, associate professor of Historical Theological and Social Ethics, Knox Theological Seminary; Spokesman for the Interfaith Stewardship Alliance 
 Interfaith Stewardship Alliance
1 Connaughton, James L., chairman, Council on Environmental Quality 
  Council on Environmental Quality
2 Doniger, David D., policy director, Climate Center, Natural Resources Defense Council 
 Natural Resources Defense Council
3 Lomborg, Bjorn, adjunct professor, Copenhagen Consensus Center 
 Copenhagen Consensus Center
0 Jerry Hatfield, Supervisory Plant Physiologist, USDA 
 USDA
1 Heather Cooley, Senior Researcher, Pacific Institute 
 Pacific Institute
2 Tom Troxel, Director, Black 

2 Strickland, Hon. Tom, Assistant Secretary for Fish, Wildlife, and Parks, U.S. Department of the Interior 
 Department of the Interior
3 Krupp, Fred, President, Environmental Defense Fund 
 Environmental Defense Fund
4 Fehrman, Bill, President and CEO, MidAmerican Energy Company 
 Midamerican Energy
0 Jensen, Jay, Deputy Under Secretary for Natural Resources and Environment, U.S. Department of Agriculture, Washington, D.C 
 Department of Agriculture
1 Koehn, Steve, Maryland State Forester, on behalf of the National Association of State Foresters, Parkton, Maryland 
 Maryland State forester
2 Bentz, Clint, on behalf of the Oregon Tree Farm System and American ForestFoundation, Scio, Oregon 
 System and American forestfoundation
3 McPeek, Brian, North America Conservation Region Director, The NatureConservancy, Denver, Colorado 
 Nature Conservancy
4 Dr. Monaghan, Tom, on behalf of the National Alliance of Forest Owners, Starkville, Mississippi 
 National Alliance of Forest Owners
5 Nei

2 Metts, Jeff, President, Dowding Machine, Eaton Rapids, Mi 
 Dowding
3 Rodgers, David, Director For Strategic Planning And Analysis, Office Of Energy Efficiency And Renewable Energy, Department Of Energy 
 Department of Energy
4 Savitz, Maxine, Ph.D., Honeywell, Inc. (Ret.), Los Angeles, Ca 
 Honeywell
5 Zepponi, David, President, Northwest Food Processors Association, Portland, Or 
 Northwest Food Processors Association
0 Panel One Dan Elwell, Assistant Administrator for Aviation Policy, Planning, and Environment, U.S. Federal Aviation Administration 
 Federal Aviation Administration
1 Bob Meyers, Principal Deputy Assistant Administrator for the Office of Air and Radiation, U.S. Environmental Protection Agency 
 Environmental Protection Agency
2 Panel Two Jim May, President and CEO, Air Transport Association 
 Air Transport Association
3 Tom Windmuller, Senior Vice President, International Air Transport Association 
 International Air Transport Association
4 Deron Lovaas, Transportat

...

0 Eckhart, Michael T., President, American Council On Renewable Energy (ACORE) 
 American Council on Renewable Energy
1 Katzer, Dr. James R., The Laboratory for Energy and the Environment, Massachusetts Institute of Technology (MIT) 
 Massachusetts Institute of Technology
2 Preli, Dr. Frank, Vice President of Engineering, UTC Power 
 Utc Power
3 Prindle, William, Acting Executive Director, American Council for an Energy-Efficient Economy (ACEEE) 
 American Council for An Energy-Efficient Economy
4 Sridhar, K.R., Principal Co-Founder/CEO, Bloom Energy 
 Bloom Energy
0 John Ward, vice president, Headwaters Incorporated 
 Headwaters Incorporated
1 Donald W. Maley, Jr., vice president, Leucadia International Corporation 
 Leucadia International Corp
2 Brian Foody, chief executive officer, Iogen Corporation 
 IOGEN Corp
3 Scott Hughes, director, government affairs, National Biodiesel Board 
 National Biodiesel Board
4 Phil Lampert, executive director, National Ethanol Vehicle Coalition 
 Na

11 Richard Richels, Technical Executive For Global Climate Change Research, Electric Power Research Institute 
 Electric Power Research Institute
12 Jason Grumet, Executive Director, National Commission On Energy Policy 
 Commission on Energy
13 Samuel Wolfe, Chief Counsel, New Jersey Board Of Public Utilities 
 Board of Public Utilities
14 William Pizer, Senior Fellow, Resources For The Future 
 Resources for the Future
15 Kateri Callahan, President, Alliance To Save Energy 
 Alliance to Save Energy
16 Michael Bradley, Executive Director, Clean Energy Group 
 Clean Energy Group
17 Michael Morris, Chairman Of The Board Of Directors, Edison Electric Institute 
 Edison Electric Institute
18 Fred Krupp, President, Environmental Defense 
 Environmental Defense
19 Paul Bailey, Director, Generators For Clean Air 
 Generators for Clean Air
20 Craig Montesano, Director Of Governmental Affairs, National Mining Association 
 National Mining Association
21 Kirk Johnson, Executive Director Of Envi

32 Reverend C. Douglas Smith, Executive Director, Virginia Interfaith Center for Public Policy 
 Virginia Interfaith Center for Public Policy
33 Howard Gruenspecht, Acting Administrator, United States Energy Information Agency 
 Energy Information Agency
34 Dan W. Reicher, Director, Climate Change and Energy Initiatives, Google 
 Google
35 Dian M. Grueneich, Commissioner, California Public Utilities Commission 
 Public Utilities Commission
36 James L. Robo, President and Chief Operating Officer, FPL Group 
 FPL Group
37 Gregory P. Kunkel, Vice President of Environmental Affairs, Tenaska, Inc 
 Tenaska
38 David G. Hawkins, Director of Climate Programs, Natural Resources Defense Council 
 Natural Resources Defense Council
39 Eugene M. Trisko, On Behalf of the United Mine Workers of America 
 United Mine Workers
40 Jonathan Briggs, Regional Director of the Americas, Hydrogen Energy International, LLC 
 Hydrogen Energy International LLC
41 James Kerr, Partner, McGuire Woods LLP, Former Com

2 Nunez, Hon. Fabian, speaker, California State Assembly 
 California State Assembly
3 Adkins, Dennis, chairman, House Committee on Energy and Technology, Oklahoma State House 
 Committee on Energy
4 Harvey, Hon. Ted, Senator, Colorado State Senate 
 Colorado State Senate
5 Nickels, Hon. Greg, Mayor, City of Seattle, WA 
 City of Seattle, WA
6 Cownie, Hon. Frank, Mayor, City of Des Moines, IA 
 City of Des Moines, IA
7 Homrighausen, Hon. Richard P., Mayor, City of Dover, OH 
 Mayor, City of Dover
0 Box, Matthew, Vice-Chairman, Southern Ute Indian Tribe 
 Ute Indian Tribe
1 Lee, Robert G., Ph.D., Professor, Sociology of Natural Resources, University of Washington 
 University of Washington
2 Marshall, Clifford Lyle, Chairman, Hoopa Valley Tribe 
 Hoopa Valley Tribe
3 Propst, Luther, Executive Director, Sonoran Institute 
 Sonoran Institute
4 Schweitzer, Hon. Brian, Governor, State of Montana 
 State of Montana
5 Vaagen, Russell C., Vice President, Vaagen Bros. Lumber Inc 
 Lumber Inc
6 

2 Christopher Cabaldon, Mayor, City of West Sacramento, California, and Transportation Vice Chair, Sacramento Area Council of Governments 
 City of West Sacramento, CA
3 Randal O'Toole, Senior Fellow, The Cato Institute 
 Cato Institute
4 Ernest Tollerson, Director, Policy and Media Relations, New York State Metropolitan Transportation Authority 
 York State Metropolitan
0 Dr. Sylvia Earle, Explorer-in-Residence, National Geographic Society 
 National Geographic Society
1 Dr. Jane Lubchenco, Department of Zoology, Oregon State University 
 Oregon State University
2 Dr. Joan Kleypas, National Center for Atmospheric Research, Boulder, Colorado 
 National Center for Atmospheric Research
3 Ms. Vikki Spruill, President and Chief Executive Office, Ocean Conservancy 
 Ocean Conservancy
0 Hon. Paula J. Dobriansky, Under Secretary For Democracy And Global Affairs 
 Under Secretary for Democracy and Global Affairs
1 Dan Reifsnyder, Deputy Assistant Secretary, Bureau Of Oceans Environment Science

In [30]:
export_df

,congress,committee,year,date,hearing_title,hearing_id,witness,affiliation,opensectrets_industry,opensectrets_sector,category,subcategory,subsubcategory,state_research,contrarian_organisation,contrarian
0,111,Committee on Energy and Commerce,2009,"march 18, 2009",competitiveness and climate policy: avoiding l...,111hhrg67814,"John McMackin, Jr., Williams and Jenson, PLLC,...",Energy Intensive Manufacturers,Misc Manufacturing & Distributing,Misc Business,Industry Allies,Miscellaneous Business,Misc Business,0,None,Other witness
1,111,Committee on Energy and Commerce,2009,"march 18, 2009",competitiveness and climate policy: avoiding l...,111hhrg67814,"Martin McBroom, Director, Federal Environmenta...",American Electric Power,Electric Utilities,Energy & Natural Resources,Industry Allies,Fossil Fuel Industry,Electric Utilities,0,None,Other witness
2,111,Committee on Energy and Commerce,2009,"march 18, 2009",competitiveness and climate policy: avoiding l...,111hhrg67814,"Paul Cicio, Industrial Energy Consumers of Ame...",Industrial Energy Consumers of America,Misc Manufacturing & Distributing,Misc Business,Contrarians,Contrarians,Misc Business,0,industrial energy consumers of america,Contrarian
3,111,Committee on Energy and Commerce,2009,"march 18, 2009",competitiveness and climate policy: avoiding l...,111hhrg67814,"Margo Thorning, Ph.D., Senior Vice President a...",American Council for Capital Formation,Business Associations,Misc Business,Contrarians,Contrarians,Misc Business,0,American Council for Capital Formation,Contrarian
4,111,Committee on Energy and Commerce,2009,"march 18, 2009",competitiveness and climate policy: avoiding l...,111hhrg67814,"Richard D. Morgenstern, Senior Fellow, Resourc...",Resources for the Future,Environment,Ideological/Single-Issue,Non-Governmental Organisations,Environmental Allies,Environment,0,None,Other witness
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1665,109,Committee on Environment and Public Works,2006,"december 6, 2006",s. hrg. 109-1077 examining climate change and ...,109shrg52324,"Carter, R.M., Ph.D., Marine Geophysical Labora...",James Cook University,Education,Other,Scientists,Scientists,Education,0,None,Other witness
1666,109,Committee on Environment and Public Works,2006,"december 6, 2006",s. hrg. 109-1077 examining climate change and ...,109shrg52324,"Deming, David, Ph.D., University of Oklahoma, ...",University of Oklahoma,Education,Other,Contrarians,Contrarians,Education,0,None,Contrarian
1667,109,Committee on Environment and Public Works,2006,"december 6, 2006",s. hrg. 109-1077 examining climate change and ...,109shrg52324,"Gainor, Dan, The Boone Pickens Free Market Fel...",Business & Media Institute,Republican/Conservative,Ideological/Single-Issue,Contrarians,Contrarians,Republican/Conservative,0,None,Contrarian
1668,109,Committee on Environment and Public Works,2006,"december 6, 2006",s. hrg. 109-1077 examining climate change and ...,109shrg52324,"Oreskes, Naomi, professor, Department of Histo...",University of California,Education,Other,Scientists,Scientists,Education,0,None,Other witness


In [35]:
outfile = open('Analysis/witnesses.csv', 'w')
export_df.to_csv(outfile, index = False, header = True, sep = '\t', encoding = 'utf-8')
outfile.close()

<br>

## 6) Export the data at the hearing level

In [33]:
# Create a dataframe of the hearings
columns = ['date', 'year', 'congress', 'hearing_title', 'hearing_id', 'committee']
hearings = pd.DataFrame(columns=columns)
for text in t.texts:
    hearings = hearings.append({'date' : text['date'], 'year': text['year'] , 'congress': text['congress'], 'committee': text['committee'],
                                'hearing_title': text['title'], 'hearing_id': text['identifier']}, ignore_index=True)
hearings

,date,year,congress,hearing_title,hearing_id,committee
0,"march 18, 2009",2009,111,competitiveness and climate policy: avoiding l...,111hhrg67814,Committee on Energy and Commerce
1,"june 25, 2008",2008,110,national security implications of global clima...,110hhrg62523,Permanent Select Committee on Intelligence
2,"september 6, 2007",2007,110,the future of coal under carbon cap and trade,110hhrg58149,Select Committee on Energy Independence and Gl...
3,"december 4, 2003",2003,108,review of non-oil and gas research activities ...,108hhrg90674,Committee on Science
4,"february 28, 2007",2007,110,energy and tax policy,110hhrg35774,Committee on Ways and Means
...,...,...,...,...,...,...
252,"june 9, 2010",2010,111,s. hrg. 111-1241 legislative hearing to consid...,111shrg23569,Committee on Environment and Public Works
253,"november 14, 2005",2005,109,s. hrg. 109-879 u.s.-international climate cha...,109shrg33730,Committee on Foreign Relations
254,"march 18, 2009",2009,111,atc modernization and nextgen near-term achiev...,111hhrg49983,Committee on Transportation and Infrastructure
255,"march 3, 2010",2010,111,the department of energy fiscal year 2011 rese...,111hhrg55839,Committee on Science and Technology


In [36]:
# Save hearings dataframe
outfile = open('Analysis/hearings.csv', 'w')
hearings.to_csv(outfile, index = False, header = True, sep = '\t', encoding = 'utf-8')
outfile.close() # Last completed on Mar 04, 2022